<a href="https://colab.research.google.com/github/Gayeon6423/BusinessAI-Capston/blob/main/5_Modeling_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Timeline

### 1. Import library
### 2. Load Data
### 3. Sentiment Analysis

### 1. Import library

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")
# 모듈 설치
# !pip install konlpy
!pip install transformers
!pip install tensorflow_addons
# 데이터 처리 모듈
import pandas as pd
import numpy as np
import copy
import re
import math
import json
import requests
import urllib.request
from tqdm import tqdm
import datetime
import time
import os
# 텍스트 관련 모듈
# from konlpy.tag import Okt
# okt = Okt()
# from konlpy.tag import *
# import nltk
# 전처리 모듈
from sklearn.preprocessing import MinMaxScaler
#시각화 모듈
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns
# 모델링 관련 모듈
import tensorflow_addons as tfa
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, \
                            roc_auc_score, confusion_matrix, classification_report, \
                            matthews_corrcoef, cohen_kappa_score, log_loss

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.3/612.3 kB 7.2 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


- Checking GPU Working

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
  print("GPU 작동 중")
  mirrored_strategy = tf.distribute.MirroredStrategy()
else:
  print("GPU 미작동 중")

GPU 작동 중


### 2. Load Data

In [ ]:
class LoadGoogleDriveData():
  def __init__(self, data = None):
    self.data = data

  def loadData(self, file_path: str, file_name_extension,
               columnTF: bool, unicode: str) -> pd.DataFrame():
    self.data = pd.read_csv(os.path.join(file_path + file_name_extension),
                            index_col = columnTF,
                            sep = ",",
                            na_values = "NaN",
                            encoding = unicode)
    return self.data

  def loadTxTData(self, file_path: str, file_name_extension,
               columnTF: bool, unicode: str) -> pd.DataFrame():
    self.data = pd.read_csv(os.path.join(file_path + file_name_extension),
                            index_col = columnTF,
                            sep = "|",
                            na_values = "NaN",
                            encoding = unicode)
    return self.data

  def loadExcelData(self, file_path: str, file_name_extension,
               columnTF: bool) -> pd.DataFrame():
    self.data = pd.read_excel(os.path.join(file_path + file_name_extension),
                              index_col = columnTF)
    return self.data

  # 용량이 큰 csv 파일 읽어오기(fopen - fread와 유사한 방식)
  def loadDataWithChunking(self, file_path: str, file_name_extension,
                           chunking_row_num: int, columnTF: bool, unicode: str) -> pd.DataFrame():
    chunkdata = pd.read_csv(os.path.join(file_path + file_name_extension),
                            chunksize = chunking_row_num,
                            index_col = columnTF,
                            sep = ",",
                            na_values = "NaN",
                            encoding = unicode)
    self.data = list(chunkdata)
    self.data = pd.concat(self.data)

    return self.data

In [ ]:
mountInstance = LoadGoogleDriveData()

- load financial sentiment data

In [ ]:
fin_sentiment_dataset = mountInstance.loadData(
            file_path = '/content/drive/MyDrive/산업 AI 캡스톤/DATA/',
            file_name_extension = "fin_sentiment_dataset.csv",
            unicode = 'utf-8-sig', columnTF = False)

### 3. Sentiment Analysis



```
- financial_phrasebank : https://huggingface.co/datasets/financial_phrasebank
- 한국어 경제 뉴스 기사 감정 분류 github : https://github.com/park-gb/financial-news-sentiment-classifier
- 한국어 경제 뉴스 기사 감정 분류 blog : https://heytech.tistory.com/394
```



In [ ]:
# Import KLUE BERT-base Model & Tokenizer From Hugging Face
MODEL_NAME = "klue/bert-base"
model = TFBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3, from_pt=True)
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

# Train, Target 데이터 분리
X_data = fin_sentiment_dataset['kor_sentence']
y_data = fin_sentiment_dataset['labels']

# X_train, X_test, y_train, y_target 데이터 분리
TEST_SIZE = 0.2 # Train: Test = 8 :2 분리
RANDOM_STATE = 42
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data,
                                                    test_size = TEST_SIZE,
                                                    random_state = RANDOM_STATE,
                                                    stratify = y_data)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


- BERT 입력용 데이터 포맷 변경

In [ ]:
# 입력 데이터(문장) 길이 제한
MAX_SEQ_LEN = 64

def convert_data_X(X_data):
    # BERT 입력으로 들어가는 token, mask, segment 저장용 리스트
    tokens, masks, segments = [], [], []

    for X in tqdm(X_data):
        # token: 입력 문장 토큰화
        token = tokenizer.encode(X, truncation = True, padding = 'max_length', max_length = MAX_SEQ_LEN)
        # Mask: 토큰화한 문장 내 패딩이 아닌 경우 1, 패딩인 경우 0으로 초기화
        num_zeros = token.count(0)
        mask = [1] * (MAX_SEQ_LEN - num_zeros) + [0] * num_zeros
        # segment: 문장 전후관계 구분: 오직 한 문장이므로 모두 0으로 초기화
        segment = [0]*MAX_SEQ_LEN
        tokens.append(token)
        masks.append(mask)
        segments.append(segment)

    # numpy array로 저장
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)

    return [tokens, masks, segments]

def convert_data_y(y_data):
    # BERT 입력으로 들어가는 target 저장용 리스트
    targets = []
    for y in tqdm(y_data):
        targets.append(y)
    # numpy array로 저장
    targets = np.array(targets)

    return targets

In [ ]:
# train 데이터를 Bert의 Input 타입에 맞게 변환
train_x = convert_data_X(X_train)
train_y = convert_data_y(y_train)

# test 데이터를 Bert의 Input 타입에 맞게 변환
test_x = convert_data_X(X_test)
test_y = convert_data_y(y_test)

100%|██████████| 966/966 [00:00<00:00, 1218922.28it/s]


- BERT 기반 감정 분류 모델 생성

In [ ]:
# token, mask, segment 입력 정의
token_inputs = tf.keras.layers.Input((MAX_SEQ_LEN,), dtype = tf.int32, name = 'input_word_ids')
mask_inputs = tf.keras.layers.Input((MAX_SEQ_LEN,), dtype = tf.int32, name = 'input_masks')
segment_inputs = tf.keras.layers.Input((MAX_SEQ_LEN,), dtype = tf.int32, name = 'input_segment')
bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

# BERT 출력 정의
bert_output = bert_outputs[0]

# 감정 분류 모델 컴파일
DROPOUT_RATE = 0.5
NUM_CLASS = 3
dropout = tf.keras.layers.Dropout(DROPOUT_RATE)(bert_output)

# Multi-class classification 문제이므로 activation function은 softmax로 설정
sentiment_layer = tf.keras.layers.Dense(NUM_CLASS, activation='softmax', kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev=0.02))(dropout)
sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_layer)

# 옵티마이저 Rectified Adam 하이퍼파리미터 조정
OPTIMIZER_NAME = 'RAdam'
LEARNING_RATE = 5e-5
TOTAL_STEPS = 10000
MIN_LR = 1e-5
WARMUP_PROPORTION = 0.1
EPSILON = 1e-8
CLIPNORM = 1.0
optimizer = tfa.optimizers.RectifiedAdam(learning_rate = LEARNING_RATE,
                                          total_steps = TOTAL_STEPS,
                                          warmup_proportion = WARMUP_PROPORTION,
                                          min_lr = MIN_LR,
                                          epsilon = EPSILON,
                                          clipnorm = CLIPNORM)
# 감정분류 모델 컴파일
sentiment_model.compile(optimizer = optimizer,
                        loss = tf.keras.losses.SparseCategoricalCrossentropy(),
                        metrics = ['accuracy'])
# Early stoping 적용
MIN_DELTA = 1e-3
PATIENCE = 5
early_stopping = EarlyStopping(
    monitor = "val_accuracy",
    min_delta = MIN_DELTA,
    patience = PATIENCE)

# 최고 성능의 모델 파일을 저장할 이름과 경로 설정
#BEST_MODEL_NAME = '/content/drive/MyDrive/산업 AI 캡스톤/model/best_model.h5'
BEST_MODEL_NAME = 'best_model.h5'
model_checkpoint = ModelCheckpoint(
    filepath = BEST_MODEL_NAME,
    monitor = "val_loss",
    mode = "min",
    save_best_only = True, # 성능 향상 시에만 모델 저장
    verbose = 1)

# Early stop과 Model checkpoint를 콜백 함수로 설정
callbacks = [early_stopping, model_checkpoint]

- 감정 분류 모델 학습

In [ ]:
EPOCHS = 100
BATCH_SZIE = 32
#model.compile(optimizer,loss)
sentiment_model.fit(train_x, train_y,
                    epochs = EPOCHS,
                    shuffle = True,
                    batch_size = BATCH_SZIE,
                    validation_data = (test_x, test_y),
                    callbacks = callbacks)

Epoch 1/100
121/121 [==============================] - ETA: 0s - loss: 1.0948 - accuracy: 0.4284
Epoch 1: val_loss improved from inf to 1.08756, saving model to best_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


121/121 [==============================] - 106s 554ms/step - loss: 1.0948 - accuracy: 0.4284 - val_loss: 1.0876 - val_accuracy: 0.5880
Epoch 2/100
121/121 [==============================] - ETA: 0s - loss: 1.0712 - accuracy: 0.5006
Epoch 2: val_loss improved from 1.08756 to 1.04748, saving model to best_model.h5
121/121 [==============================] - 67s 551ms/step - loss: 1.0712 - accuracy: 0.5006 - val_loss: 1.0475 - val_accuracy: 0.5983
Epoch 3/100
121/121 [==============================] - ETA: 0s - loss: 1.0210 - accuracy: 0.5602
Epoch 3: val_loss improved from 1.04748 to 0.97557, saving model to best_model.h5
121/121 [==============================] - 67s 551ms/step - loss: 1.0210 - accuracy: 0.5602 - val_loss: 0.9756 - val_accuracy: 0.7267
Epoch 4/100
121/121 [==============================] - ETA: 0s - loss: 0.9591 - accuracy: 0.6149
Epoch 4: val_loss improved from 0.97557 to 0.90573, saving model to best_model.h5
121/121 [==============================] - 68s 564ms/step - 

- 감정 분류 모델 성능 평가

In [ ]:
# 최고 성능의 모델 불러오기
sentiment_model_best = tf.keras.models.load_model(BEST_MODEL_NAME,custom_objects={'TFBertForSequenceClassification': TFBertForSequenceClassification})
# 모델이 예측한 라벨 도출
predicted_value = sentiment_model_best.predict(test_x)
predicted_label = np.argmax(predicted_value, axis = 1)
# 평가지표
accuracy_score_v = round(accuracy_score(y_test, predicted_label), 3) # Accuracy
precision_score_v = round(precision_score(y_test, predicted_label, average="weighted"), 3) # Precision
recall_score_v = round(recall_score(y_test, predicted_label, average="weighted"), 3) # Recall
f1_score_v = round(f1_score(y_test, predicted_label, average="weighted"), 3) # F1 Score
roc_auc_score_v = round(roc_auc_score(y_test, predicted_value, average="weighted", multi_class="ovr"), 3) # ROC AUC Score
cohen_kappa_score_v = round(cohen_kappa_score(y_test, predicted_label), 3) # Cohen's Kappa Score
matthews_corrcoef_v = round(matthews_corrcoef(y_test, predicted_label), 3) # Matthew's Correlation Coefficient
log_loss_v = round(log_loss(y_test, predicted_value), 3) # Log Loss
# Classification 종합 평가지표 데이터프레임으로 저장
METRIC_FILE = f'/content/drive/MyDrive/산업 AI 캡스톤/DATA/Sentiment_Metric/Sentiment_metric_KLUEBERT.csv'
metric_total = pd.DataFrame({'PLM': MODEL_NAME,'Optimizer': OPTIMIZER_NAME,'Accuracy': accuracy_score_v,
                             'Precision': precision_score_v,'Recall': recall_score_v,'F1_score': f1_score_v,
                             'ROC_AUC_score': roc_auc_score_v,'Cohen_kappa_coef': cohen_kappa_score_v,
                             'Matthews_corrcoef': matthews_corrcoef_v,'Log_loss': log_loss_v},index = ['-'])
metric_total.to_csv(METRIC_FILE)
print('accuracy_score_v : ', accuracy_score_v)
print('accuracy_score : ', accuracy_score_v)
print('roc_auc_score : ', roc_auc_score_v)

31/31 [==============================] - 7s 140ms/step
accuracy_score_v :  0.837
accuracy_score :  0.837
roc_auc_score :  0.913


- 2023년1월 뉴스 기사 감정 스코어 예측



```
file_path = '/content/drive/MyDrive/산업 AI 캡스톤/DATA/Preprocessing_Data/Kospi_Preprocessing_News_Data/'
file_name = "news_preprocess_202301_kospi.csv"
news_preprocess_202301_kospi = mountInstance.loadData(file_path = file_path, file_name_extension = file_name, unicode='utf-8-sig', columnTF=False)

X_test =  news_preprocess_202301_kospi['preprocess_context'] # 네이버 뉴스 기사
X_test = convert_data_X(X_test)
predicted_value = sentiment_model_best.predict(X_test) # 중립, 긍정, 부정
predicted_label = np.argmax(predicted_value, axis = 1)
# 감정 스코어 변환
col_name = ['neural','positive','negative']
predicted_value_df = pd.DataFrame(predicted_value,columns = col_name)
predicted_value_df['score'] = predicted_value_df['positive'] - predicted_value_df['negative']
news_preprocess_202301_kospi = news_preprocess_202301_kospi[['pubdate','preprocess_context']]
news_preprocess_202301_kospi['sentiment_score'] = predicted_value_df['score']
# pubdate기준으로 그룹화하고 'sentiment_score'의 평균을 계산
sentiment_202301_kospi = news_preprocess_202301_kospi.groupby('pubdate')['sentiment_score'].mean().reset_index()
sentiment_202301_kospi.head(20)
```



In [ ]:
def sentiment_scoring(start_date, end_date, keyword):
    # sentiment score dataframe
    daily_sentiments_tmp = pd.DataFrame(columns = ['Date','Positive','Negative','Neutral'])
    daily_sentiment = pd.DataFrame(columns = ['Date','Positive','Negative','Neutral'])

    # Read the Month News Data
    for month in pd.date_range(start_date, end_date, freq = 'M'):
        file_path = f'/content/drive/MyDrive/산업 AI 캡스톤/DATA/Preprocessing_Data/{keyword}_Preprocessing_News_Data/'
        file_name = f"news_preprocess_{month.strftime('%Y%m')}_{keyword}.csv"
        month_news_data = mountInstance.loadData(file_path = file_path, file_name_extension = file_name, unicode='utf-8-sig', columnTF=False)

        # 뉴스 데이터 훈련데이터의 test 데이터로 설정
        X_test = month_news_data['preprocess_context']
        X_test = convert_data_X(X_test)
        predicted_value = sentiment_model_best.predict(X_test) # 중립, 긍정, 부정 예측

        # Sentiment Score 계산
        col_name = ['neural','positive','negative']
        predicted_value_df = pd.DataFrame(predicted_value,columns = col_name)
        predicted_value_df['score'] = predicted_value_df['positive'] - predicted_value_df['negative']

        # DataFrame 생성 및 저장
        month_news_data = month_news_data[['pubdate','preprocess_context']]
        month_news_data['sentiment_score'] = predicted_value_df['score']

        # pubdata 기준으로 그룹화하고 'sentiment_score'의 평균을 계산
        ## 데이터 수집 완료 후 에는 장 시작 시간, 장 마감 시간도 고려해서 평균 계산 예정
        daily_sentiments = month_news_data.groupby('pubdate')['sentiment_score'].mean().reset_index()
        daily_sentiments.reset_index(drop=True).rename(columns = {'pubdate':'Date'})
        # Save File path
        file_path = '/content/drive/MyDrive/산업 AI 캡스톤/DATA/Sentiment_Score_Data/'
        daily_sentiments.to_csv(file_path + f'Sentiment_Score_{keyword}_{month.strftime("%Y%m")}_KLUEBERT.csv',index=False)
        return daily_sentiments

- Kospi Sentiment Analysis

In [ ]:
# 뉴스 본문 기반 감정분석
start_date = '2023-01'
end_date = '2023-07'
keyword = "Kospi"
sentiment_scoring(start_date, end_date, keyword)

- Kosdaq Sentiment Analysis

In [ ]:
start_date = '2023-01'
end_date = '2023-02'
keyword = "Kosdaq"
sentiment_scoring(start_date, end_date, keyword)

In [ ]:
# 결과
file_path = '/content/drive/MyDrive/산업 AI 캡스톤/DATA/Sentiment_Score_Data/'
file_name = 'Sentiment_Score_Kospi_202301_KLUEBERT.csv'
Sentiment_Score_Kospi_202301 = mountInstance.loadData(file_path = file_path, file_name_extension = file_name, unicode='utf-8-sig', columnTF=False)
Sentiment_Score_Kospi_202301.head(5)

,pubdate,sentiment_score
0,20230103,0.073552
1,20230104,0.100598
2,20230105,0.068558
3,20230106,0.033046
4,20230109,0.067463
